In [1]:
%autosave 0
import sys
sys.dont_write_bytecode = True
from methods import *

Autosave disabled


# set parameters

In [2]:
# set parent path to directory where data is stored (with observations as a subdirectory in this parent dir)
data_dir = '/home/tim/Dropbox/data/'


# **** the parameters below are static values for your reference, select from these to use in below cells

# will display data from these sensors in subplots later, in order from top-to-bottom left-to-right
# bounding boxes will be drawn on the previous sensor in list
sensor_names_all = [
    'SceneV1', # forward RGB scene (sunny, no weather) 144x256 resolution
    'BoxesV1', # ground truth bounding boxes of all objects in scene, will be drawn onto sensor above this
    'SegmentationV1', # ground truth color forward segementation 144x256 resolution
    'MasksV1', # ground truth dictionary of boolean array masks for each object in scene
    'DepthV1', # ground truth forward depth 144x256 resolution
    'DepthV2', # ground truth forward depth 36x64 resolution 
    'SceneV3', # forward RGB rain 144x256 resolution
    'SceneV4', # forward RGB snow 144x256 resolution
    'SceneV5', # forward RGB fog 144x256 resolution
]

# id_names are a unique identifier to tag specific subsets or versions of a data_dict
id_names_all = [
    'alpha', # most current stable data
    'beta', # experimental data -- will update to alpha when finished
    'demo', # quickly collected data to show
]

# these are valid ranges of coordinates that you can use
    # train is top half, test is bottom half, all is both halves
x_vals_train_blocks = [x for x in range(-60, 101, 2)]
x_vals_test_blocks = [x for x in range(-120, 1, 2)]
x_vals_all_blocks = [x for x in range(-120, 101, 2)]
y_vals_all_blocks = [y for y in range(-140, 141, 2)]
x_vals_train_airsimnh = [x for x in range(-10, 241, 2)]
x_vals_test_airsimnh = [x for x in range(-240, 11, 2)]
x_vals_all_airsimnh = [x for x in range(-240, 241, 2)]
y_vals_all_airsimnh = [y for y in range(-240, 241, 2)]
z_vals_2d, z_vals_3d = [-4], [-4] #[-16, -12, -8, -4] # working on z-vals outside of -4
yaw_vals_all = [0, 1, 2, 3]
#t_vals_all = [0] # working on temporal values to add to dictionary 

# rename titles on subplots in animation
sensor_psuedonames = {
    'DepthV1':'Depth', # forward depth 144x256 resolution
    'DepthV2':'Depth', # forward depth 144x256 resolution
    'SceneV1':'Sunny', # forward RGB 144x256 resolution
    'SceneV3':'Rain', # forward RGB rain 144x256 resolution
    'SceneV4':'Snow', # forward RGB snow 144x256 resolution
    'SceneV5':'Fog', # forward RGB fog 144x256 resolution
}

In [3]:
rdir = f'{data_dir}observations/SceneV1/AirSimNH/'
fnames = os.listdir(rdir)
for fname in fnames:
    if 'data_dict' in fname:
        print(fname)
        data_dict = pk_read(rdir+fname)
        for x in data_dict:
            for y in data_dict[x]:
                for z in data_dict[x][y]:
                    if len(data_dict[x][y][z]) != 4:
                        print(x, y, z)

data_dict__alpha_1_-112_-48_2_-176_-112_2_-4_0_4.p
data_dict__alpha_1_208_241_2_-48_16_2_-4_0_4.p
data_dict__alpha_1_-176_-112_2_-240_-176_2_-4_0_4.p
data_dict__alpha_1_16_80_2_208_241_2_-4_0_4.p
data_dict__alpha_1_80_144_2_-176_-112_2_-4_0_4.p
data_dict__alpha_1_80_144_2_16_80_2_-4_0_4.p
data_dict__alpha_1_80_144_2_80_144_2_-4_0_4.p
data_dict__alpha_1_-176_-112_2_16_80_2_-4_0_4.p
data_dict__alpha_1_80_144_2_-48_16_2_-4_0_4.p
data_dict__alpha_1_-48_16_2_80_144_2_-4_0_4.p
data_dict__alpha_1_-176_-112_2_-112_-48_2_-4_0_4.p
data_dict__alpha_1_80_144_2_144_208_2_-4_0_4.p
data_dict__alpha_1_144_208_2_16_80_2_-4_0_4.p
data_dict__alpha_1_208_241_2_208_241_2_-4_0_4.p
data_dict__alpha_1_-240_-176_2_-112_-48_2_-4_0_4.p
data_dict__alpha_1_208_241_2_-176_-112_2_-4_0_4.p
data_dict__alpha_1_208_241_2_80_144_2_-4_0_4.p
data_dict__alpha_1_16_80_2_-48_16_2_-4_0_4.p
data_dict__alpha_1_144_208_2_-112_-48_2_-4_0_4.p
data_dict__alpha_1_80_144_2_-240_-176_2_-4_0_4.p
data_dict__alpha_1_-48_16_2_-240_-176_2_-

# fetch train and test data

In [4]:
# set coordinate ranges
x_vals = x_vals_train_airsimnh
y_vals = y_vals_all_airsimnh
z_vals = z_vals_2d
yaw_vals = yaw_vals_all
airsim_map = 'AirSimNH'
sensor_names = [
    'SceneV1',
    'DepthV1',
]
sensor_psuedonames = {
    'SceneV1':'rgb',
    'DepthV1':'depth',
}
id_names = ['alpha'] # only use stable data
resolution=[144,256] # height, width of observation arrays (images)

# fetch data from coordinates
location_data, location_animation = data_at_coordinates(data_dir, airsim_map, sensor_names, id_names, x_vals, y_vals, z_vals, yaw_vals, 
                    make_animation=False, return_data=True, ncols=2, additional_subplots=0, resolution=resolution, sensor_psuedonames=sensor_psuedonames)

# display results
print('Train Data')
display_data_results(location_data, location_animation, sensor_psuedonames)

# write results
out_dir = f'{data_dir}prepared_datasets/rgb_to_depth_v1/'
os.makedirs(out_dir, exist_ok=True)
for sensor_name in sensor_names:
    data = location_data['observations'][sensor_name]
    out_path = f'{out_dir}train_{sensor_psuedonames[sensor_name]}.npy'
    np.save(out_path, data)
np.save(f'{out_dir}train_coordinates.npy', location_data['coordinates'])

Train Data
collected 99480 many data points!
rgb has shape of (99480, 3, 144, 256)
depth has shape of (99480, 1, 144, 256)


In [5]:
# set coordinate ranges
x_vals = x_vals_test_airsimnh
y_vals = y_vals_all_airsimnh
z_vals = z_vals_2d
yaw_vals = yaw_vals_all
airsim_map = 'AirSimNH'
sensor_names = [
    'SceneV1',
    'DepthV1',
]
sensor_psuedonames = {
    'SceneV1':'rgb',
    'DepthV1':'depth',
}
id_names = ['alpha'] # only use stable data
resolution=[144,256] # height, width of observation arrays (images)

# fetch data from coordinates
location_data, location_animation = data_at_coordinates(data_dir, airsim_map, sensor_names, id_names, x_vals, y_vals, z_vals, yaw_vals, 
                    make_animation=False, return_data=True, ncols=2, additional_subplots=0, resolution=resolution, sensor_psuedonames=sensor_psuedonames)

# display results
print('Train Data')
display_data_results(location_data, location_animation, sensor_psuedonames)

# write results
out_dir = f'{data_dir}prepared_datasets/rgb_to_depth_v1/'
os.makedirs(out_dir, exist_ok=True)
for sensor_name in sensor_names:
    data = location_data['observations'][sensor_name]
    out_path = f'{out_dir}test_{sensor_psuedonames[sensor_name]}.npy'
    np.save(out_path, data)
np.save(f'{out_dir}test_coordinates.npy', location_data['coordinates'])

Train Data
collected 95608 many data points!
rgb has shape of (95608, 3, 144, 256)
depth has shape of (95608, 1, 144, 256)
